## Lesson 8 : Convolution networks

### But FIRST - finish up Collaborative filtering!

I included notes on the rest of Chapter 8  in the previous lesson because I had no idea he was going to continue on in this lesson. 

For reference he goes over:

* Creating your own embedding layer

* Intepreting embeddings and biases 

* Using fastai's `collab_learner` implementation

* Embedding distances 

* Deep learning for collab filtering


### Embeddings for NLP (30:30)

* Discussion of how embeddings are used words, which are just a large categorical variable (the vocab)

* The neural net only sees the embeddings, which it learns.


### Embeddings for Tabular

* As mentioned (i think?) before we can also turn any categorical into an embedding, for example those that often appear in tabular data. 

* This is done in chapter 9 in the book.

* Guo, Cheng, and Felix Berkhahn. 2016. “Entity Embeddings of Categorical Variables.” arXiv. https://doi.org/10.48550/arXiv.1604.06737.

* Guo et. al. also combined deep learning embeddings with boosted trees (i.e. also feeding the embeddings into decision tree)


### CNNS

Video at 44:30

* Based partly on Chapter 13 in the book.